# Find Variance of Median
Base off example on page 49 of https://www2.census.gov/programs-surveys/nychvs/technical-documentation/variances/2017-NYCHVS-Guide-to-Estimating-Variances.pdf

In [71]:
import rpy2
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector

from rpy2.robjects import r, pandas2ri
pandas2ri.activate()

survey_package = rpackages.importr('survey')
base = rpackages.importr('base')

import numpy as np
import pandas as pd

In [72]:
import os
cwd = os.getcwd()
data_dir = cwd.rsplit('/', 1)[0] +'/data'

In [73]:
data = robjects.r(f'read.csv("{data_dir}/HVS_data.csv", header=TRUE)')


In [74]:
weights_correction = 10**5
data['fw'] = data['fw']/weights_correction

rep_weights_columns = [f'fw{i}' for i in range(1,81)]
data[rep_weights_columns] = data[rep_weights_columns]/weights_correction

In [75]:
survey_package = rpackages.importr('survey')
hu_design = survey_package.svrepdesign(variables=data.iloc[:, :188],
                          repweights=data[rep_weights_columns],
                          weights=data['fw'], combined_weights = True, type='other',
                         scale=4/80, rscales=1)

Find median age per puma

In [76]:
data['hhr3t']

1        65
2        41
3        48
4        55
5        63
         ..
13262    73
13263    47
13264    57
13265    72
13266    82
Name: hhr3t, Length: 13266, dtype: int32

In [77]:
age_per_sub_borough_area = survey_package.svyby(formula=data[['hhr3t']], by=data[['boro','cd']], 
                                           design=hu_design, FUN=survey_package.svyquantile, quantiles=np.array([.5]))

In [78]:
age_per_sub_borough_area

,boro,cd,hhr3t,se.hhr3t
1.1,1,1,50.0,1.758396
2.1,2,1,39.0,1.758396
3.1,3,1,43.0,1.758396
4.1,4,1,46.0,1.507197
5.1,5,1,53.0,1.507197
1.2,1,2,50.0,1.507197
2.2,2,2,44.0,2.009596
3.2,3,2,49.0,1.758396
4.2,4,2,48.0,2.511995
5.2,5,2,52.0,1.507197


Seems good. Do example they do and make sure our answer is the same

For tomorrow: only include renters based on following criteria  

`hu$renters = ifelse(((hu$sc116 == 2 | hu$sc116 == 3) & hu$uf26 !=
99999), 1, 0)`

In [79]:
def is_renter(row):
    if row['uf26'] == 99999:
        return False
    if row['sc116'] == 2 or row['sc116'] ==3:
        return True
    return False

In [80]:
data['renter'] = data.apply(axis=1, func=is_renter)

In [124]:
renters = data[data['renter']]
max(renters['uf26'])

7292

In [119]:
renter_design = survey_package.svrepdesign(variables=renters.iloc[:, :188],
                          repweights=renters[rep_weights_columns],
                          weights=renters['fw'], combined_weights = True, type='other',
                         scale=4/80, rscales=1)

Add gross rent variable 

`hu$gross_rent <- hu$uf26
hu$gross_rent[hu$uf26 == 99999] <- NA`

In [127]:
survey_package.svyby(formula=data[['uf26']], by=data[['a']], 
                            design=renter_design, FUN=survey_package.svyquantile, quantiles=np.array([.5]))

,a,uf26,se.uf26
a,a,2046.0,22.356751


In [108]:
median = survey_package.svyquantile(data[['uf26']], design = hu_design, quantiles=np.array([.5]))

In [109]:
median[0]

array([[2046.        , 2000.        , 2089.        ,   22.35675114]])